# Functional Test 3.2.1 - Local bridge with Shared NICs

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.2.1 for testing Shared NIC with a local bridge.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## Step 3: Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [ ]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the Test Slice

The following creates two nodes with a shared NIC each. This should be run all worker nodes regardless of type.

Two nodes with one NIC component each are created on different workers.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. 

**Be sure to try different combinations of workers**

In [ ]:
from datetime import datetime
from dateutil import tz

name1='Node1'
nic1_name='SharedNIC1'

name2='Node2'
nic2_name='SharedNIC2'

network_name='l2-bridge'

site='TACC'

# since all workers have a standard naming scheme, you can just change the worker
# to move from worker to worker
worker1=f'{site.lower()}-w1.fabric-testbed.net'
worker2=f'{site.lower()}-w2.fabric-testbed.net'

cores=10
ram=20
disk=50
slice_name=f"Slice Test 3.2.1-SharedNIC {site} {datetime.now()}"

In [ ]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=name1, site=site, host=worker1, cores=cores, ram=ram, disk=disk)
    iface1 = node1.add_component(model='NIC_Basic', name=nic1_name).get_interfaces()[0]
    
    node2 = slice.add_node(name=name2, site=site, host=worker2, cores=cores, ram=ram, disk=disk)
    iface2 = node2.add_component(model='NIC_Basic', name=nic2_name).get_interfaces()[0]
    
    # Network
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2])
 
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Step 5: Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node List

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node Details

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Interfaces

You should see 2 interfaces.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

### Shared NIC PCI Devices

Run the command <code>lspci</code> to see your Shared NIC PCI device(s).

View node1's Shared NIC - you should see a `Ethernet controller: Mellanox Technologies MT28908 Family [ConnectX-6 Virtual Function]` 

In [ ]:
command = "sudo dnf install -q -y pciutils && lspci"
try:
    node1 = slice.get_node(name=name1)  
    stdout, stderr = node1.execute(command)
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Configure interfaces, test reachability

##  Configure IP Addresses

### Pick a Subnet

Create a subnet and list of available IP addresses. You can use either IPv4 or IPv6 subnets and addresses.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Optionally, use the `node.execute()` method to show the results of adding the IP address.

In [ ]:
try:
    node1 = slice.get_node(name=name1)        
    node1_iface = node1.get_interface(network_name=network_name) 
    node1_addr = available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

### Configure Node2

Repeat the steps to add the next available IP to the second node.

In [ ]:
try:
    node2 = slice.get_node(name=name2)        
    node2_iface = node2.get_interface(network_name=network_name)  
    node2_addr = available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

### Test reachability

Test ping between interfaces, observe successful output.


In [ ]:
try:
    node1 = slice.get_node(name=name1)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

## Step 7: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")